In [674]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy.random as rnd
import random
import math
import copy

## 0. Functions

### 0.1. About chromosomes

In [675]:
"""
Generate random solution
"""
def generate_random_solution(size):
    solution = []
    chromosome = []
    sign = []
    
    # Chromosome
    current_size = 0
    while current_size < size:
        chromosome.append(random.randint(0,1))
        current_size = current_size + 1
    
    # Sign + -
    sign_candidate = [-1, 1]
    sign.append(random.choice(sign_candidate))

    # Merge solution
    solution.append(chromosome)
    solution.append(sign)
    # print(solution)

    return solution

"""
Chromosome genotype solution value function
"""

def chromosome_bin_to_decimal(chromosome): 
    #
    power = len(chromosome[0])-1    
    sum = 0
    
    #
    for idx in range(0,len(chromosome[0])):
        sum = sum + (chromosome[0][idx]) * (2**power)
        power = power - 1
    
    return sum * chromosome[1][0]


### 0.2. Objective functions

In [676]:
"""
Sphere function(objective)
input: decimal
"""
def Sphere_function(x):
    return x**2


### 0.3. About selection

In [677]:
"""
Calculate objective function and compose selection probability
"""

def calculate_obj_and_compose_selection_probability(solution_set):

    value_set = []
    for sol_idx in range(0, len(solution_set)):
        
        sum = 0
        for dim in range(0,dimension):
            sum = sum + Sphere_function(chromosome_bin_to_decimal(solution_set[sol_idx][dim]))
        value_set.append(sum)

    # print(value_set)
            

    value_set_array = np.array(value_set)
    value_set_array = 1/ value_set_array
    value_set = value_set_array.tolist()

    denominator = 0
    for i in value_set:
        denominator = denominator + i
        
    selection_prob = []
    for i in value_set:
        selection_prob.append(i/denominator)
        
    # print(selection_prob) 

    return selection_prob

### 0.4. Crossver and Mutation

In [678]:
"""
One-point crossover
: between same dimension

"""
def execute_crossover_standard(solution_1, solution_2):
    
    solution_1_tmp = copy.deepcopy(solution_1)
    solution_2_tmp = copy.deepcopy(solution_2)
    
    # Crossover
    dim = 0
    while dim < dimension:
        
        # Save current solutions
        current_sol1 = solution_1_tmp[dim][0]
        current_sol2 = solution_2_tmp[dim][0]
        # print(f'{current_sol1}\n{current_sol2}')
        
        # Copy the new solution from the current solutions
        new_sol1 = copy.deepcopy(current_sol1)
        new_sol2 = copy.deepcopy(current_sol2)
        
        # Find location for one-point crossover
        point_loc = random.randint(1,(len(current_sol1)-1))
        # print(f'point_loc: {point_loc}')
        for loc_idx in range(0, point_loc):
            
            new_sol1[loc_idx], new_sol2[loc_idx] = new_sol2[loc_idx], new_sol1[loc_idx]
        
        # print(f'{new_sol1}\n{new_sol2}')
        
        # Replace current with new solution
        solution_1_tmp[dim][0] = new_sol1
        solution_2_tmp[dim][0] = new_sol2
        
        # Update
        dim = dim + 1

    # print(f'{solution_1_tmp}\n{solution_2_tmp}')
    return solution_1_tmp, solution_2_tmp

"""
Modified crossover: crop & merge
"""
def execute_crossover_modified(solution_1, solution_2):
    
    return 1


"""
Standard mutation
: select only one
"""
def execute_mutation_standard(new_solution_1, new_solution_2):

    remember_signal = -99
    # Select
    if random.random() <= 0.5:
        mutation_target_chromosome = copy.deepcopy(new_solution_1)
        remember_signal = 1
    else:
        mutation_target_chromosome = copy.deepcopy(new_solution_2)
        remember_signal = 2
        
    # print("!: ",remember_signal)

    # Choose the location randomly along all dimension 
    dim = 0
    while dim < dimension:
        temp_chromosome = mutation_target_chromosome[dim][0]
        # print(temp_chromosome)
        
        point_loc = random.randint(0,(len(temp_chromosome)-1))
        
        # print(f'{point_loc}: {temp_chromosome[point_loc]}')
        
        if temp_chromosome[point_loc] == 0:
            temp_chromosome[point_loc] = 1
        else:
            temp_chromosome[point_loc] = 0
            
        # print(temp_chromosome)
        # print("\n")
        dim = dim + 1

    if remember_signal == 1:
        new_solution_1 = copy.deepcopy(mutation_target_chromosome)
    elif remember_signal == 2:
        new_solution_2 = copy.deepcopy(mutation_target_chromosome)
    else:
        print("error")
        
    return new_solution_1, new_solution_2

### 0.5. Calculate objective function

In [679]:
"""
Calculate objective function and judge the best solution
"""

def calculate_objective_function(solution_set):

    value_set = []
    for sol_idx in range(0, len(solution_set)):
        
        sum = 0
        for dim in range(0,dimension):
            sum = sum + Sphere_function(chromosome_bin_to_decimal(solution_set[sol_idx][dim]))
        value_set.append(sum)

    # print(value_set)
    
    return value_set

## 1. Parameters settings

In [680]:
num_pop = 100
dimension = 2
size = 7 #(-100<= X <=100 by binary)

crossover_prob = 0.8 # 0.7, 0.8, 0.9
mutation_prob = 0.1 # 0.05, 0.1, 0.15, 0.2



## 2. Generate initial population

In [681]:
"""
Example of chromosome
"""
chromosome = generate_random_solution(size)
val = chromosome_bin_to_decimal(chromosome)

print(chromosome)
print(val)

[[0, 1, 0, 0, 1, 1, 0], [1]]
38


In [682]:
"""
Generate intial population
"""
solution_set = []

successful_pop = 1
while successful_pop <= num_pop:
    
    # Generate    
    temp_solution_set = []
    success_dim = 0
    while success_dim < dimension:
        temp_chromosome = generate_random_solution(size)
        val = chromosome_bin_to_decimal(temp_chromosome)
    
        # Warm initial solution
        if (val >= -100) &  (val <= 100):
            temp_solution_set.append(temp_chromosome)
            success_dim = success_dim + 1
        else:
            pass
            # print(f'Infeasible!...')
    # Append
    solution_set.append(temp_solution_set)
    # print(f'Generated num of population: {successful_pop}')
    
    # Update
    successful_pop = successful_pop + 1
    

# for i in range(0, len(solution_set)):
    
#     print(solution_set[i])
    

## 3. Roulette-wheel selection

In [683]:
"""
Set the roulette-wheel and select two individual solutions
"""

# selection probability based on fitness values
selection_prob = calculate_obj_and_compose_selection_probability(solution_set)

# 1~Npop
num_list = list(range(1,(num_pop+1)))
selected_pop = np.random.choice(num_list, 2, p=selection_prob, replace=False)
print(selected_pop)


[ 37 100]


## 4. Crossover

In [684]:
"""
Execute crossover with crossover probability: select only two
"""
# Selected two solutions
solution_1 = solution_set[selected_pop[0]]
solution_2 = solution_set[selected_pop[1]]
print(f'current\n{solution_1}\n{solution_2}')

if random.random() <= crossover_prob:
    new_solution_1, new_solution_2 = execute_crossover_standard(solution_1, solution_2)
    print(f'new\n{new_solution_1}\n{new_solution_2}')


IndexError: list index out of range

## 5. Mutation

In [ ]:
"""
Execute muatation with crossover probability: select only one solution between two solutions
"""

print(f'current\n{new_solution_1}\n{new_solution_2}')
if random.random() <= 1: # mutation_prob
    new_solution_1, new_solution_2 = execute_mutation_standard(new_solution_1, new_solution_2)
print(f'current\n{new_solution_1}\n{new_solution_2}')

current
[[[0, 1, 1, 0, 1, 0, 1], [-1]], [[0, 1, 1, 0, 1, 0, 0], [1]]]
[[[1, 0, 0, 1, 0, 1, 1], [1]], [[0, 0, 0, 0, 1, 1, 1], [-1]]]
current
[[[0, 1, 1, 0, 1, 0, 1], [-1]], [[0, 1, 1, 0, 1, 0, 0], [1]]]
[[[1, 0, 0, 1, 1, 1, 1], [1]], [[1, 0, 0, 0, 1, 1, 1], [-1]]]


## 6. Alternate

In [ ]:
"""
Alternative I
"""

print(solution_set[selected_pop[0]])
print(solution_set[selected_pop[1]])

solution_set[selected_pop[0]] = new_solution_1
solution_set[selected_pop[1]] = new_solution_2
print("\n")

print(solution_set[selected_pop[0]])
print(solution_set[selected_pop[1]])



[[[1, 0, 0, 1, 0, 1, 1], [-1]], [[0, 0, 0, 0, 1, 1, 0], [1]]]
[[[0, 1, 1, 0, 1, 0, 1], [1]], [[0, 1, 1, 0, 1, 0, 1], [-1]]]


[[[0, 1, 1, 0, 1, 0, 1], [-1]], [[0, 1, 1, 0, 1, 0, 0], [1]]]
[[[1, 0, 0, 1, 1, 1, 1], [1]], [[1, 0, 0, 0, 1, 1, 1], [-1]]]


## Iteration test

In [718]:
"""
Generate intial population
"""
solution_set = []

successful_pop = 1
while successful_pop <= num_pop:
    
    # Generate    
    temp_solution_set = []
    success_dim = 0
    while success_dim < dimension:
        temp_chromosome = generate_random_solution(size)
        val = chromosome_bin_to_decimal(temp_chromosome)
    
        # Warm initial solution
        if (val >= -100) &  (val <= 100):
            temp_solution_set.append(temp_chromosome)
            success_dim = success_dim + 1
        else:
            pass
            # print(f'Infeasible!...')
    # Append
    solution_set.append(temp_solution_set)
    # print(f'Generated num of population: {successful_pop}')
    
    # Update
    successful_pop = successful_pop + 1
    
best_solution = []
min_best_val = 9999999999999999

iteration = 1
while iteration <= 2000:

    """
    Set the roulette-wheel and select two individual solutions
    """

    # selection probability based on fitness values
    selection_prob = calculate_obj_and_compose_selection_probability(solution_set)

    # 1~Npop
    num_list = list(range(0,num_pop))
    selected_pop = np.random.choice(num_list, 2, p=selection_prob, replace=False)
    # print(selected_pop)

    """
    Execute crossover with crossover probability: select only two
    """
    # Selected two solutions
    solution_1 = solution_set[selected_pop[0]]
    solution_2 = solution_set[selected_pop[1]]
    # print(f'current\n{solution_1}\n{solution_2}')

    if random.random() <= crossover_prob:
        new_solution_1, new_solution_2 = execute_crossover_standard(solution_1, solution_2)
        # print(f'new\n{new_solution_1}\n{new_solution_2}')

    """
    Execute muatation with crossover probability: select only one solution between two solutions
    """

    # print(f'current\n{new_solution_1}\n{new_solution_2}')
    if random.random() <= 1: # mutation_prob
        new_solution_1, new_solution_2 = execute_mutation_standard(new_solution_1, new_solution_2)
    # print(f'current\n{new_solution_1}\n{new_solution_2}')


    """
    Alternative I
    """

    # print(solution_set[selected_pop[0]])
    # print(solution_set[selected_pop[1]])

    solution_set[selected_pop[0]] = new_solution_1
    solution_set[selected_pop[1]] = new_solution_2
    # print("\n")

    # print(solution_set[selected_pop[0]])
    # print(solution_set[selected_pop[1]])
    
    
    """
    Evaluation
    """
    
    iterative_solution_set = calculate_objective_function(solution_set)

    print(f'{iteration} | Best solution {solution_set[iterative_solution_set.index(min(iterative_solution_set))]}, value: {min(iterative_solution_set)}, index: {iterative_solution_set.index(min(iterative_solution_set))}')
        
    # Update
    iteration = iteration + 1


1 | Best solution [[[0, 0, 0, 0, 0, 0, 0], [1]], [[0, 0, 0, 1, 1, 1, 0], [1]]], value: 196, index: 35
2 | Best solution [[[0, 0, 0, 0, 0, 0, 0], [1]], [[0, 0, 0, 1, 1, 1, 0], [1]]], value: 196, index: 35
3 | Best solution [[[0, 0, 0, 0, 1, 1, 1], [-1]], [[0, 0, 0, 1, 1, 1, 1], [1]]], value: 274, index: 17
4 | Best solution [[[0, 0, 0, 0, 1, 1, 1], [-1]], [[0, 0, 0, 1, 1, 1, 1], [1]]], value: 274, index: 17
5 | Best solution [[[0, 0, 0, 0, 1, 1, 1], [-1]], [[0, 0, 0, 1, 1, 1, 1], [1]]], value: 274, index: 17
6 | Best solution [[[0, 0, 0, 0, 1, 1, 1], [-1]], [[0, 0, 0, 1, 1, 1, 1], [1]]], value: 274, index: 17
7 | Best solution [[[0, 0, 0, 0, 1, 1, 1], [-1]], [[0, 0, 0, 1, 1, 1, 1], [1]]], value: 274, index: 17
8 | Best solution [[[0, 0, 0, 0, 1, 1, 1], [-1]], [[0, 0, 0, 1, 1, 1, 1], [1]]], value: 274, index: 17
9 | Best solution [[[0, 0, 0, 0, 1, 1, 1], [-1]], [[0, 0, 0, 1, 1, 1, 1], [1]]], value: 274, index: 17
10 | Best solution [[[0, 0, 0, 0, 1, 1, 1], [-1]], [[0, 0, 0, 1, 1, 1, 1], 

## Dev note
- 전반적으로 오류나는거 체크(len 범위 착각)
- selection 할때 1, 2등에 거의 몰빵해야 함 예) 50*(1/2)**(r-100)

In [722]:
for r in range(0, 100):
    print(50*(0.5)**(r))

50.0
25.0
12.5
6.25
3.125
1.5625
0.78125
0.390625
0.1953125
0.09765625
0.048828125
0.0244140625
0.01220703125
0.006103515625
0.0030517578125
0.00152587890625
0.000762939453125
0.0003814697265625
0.00019073486328125
9.5367431640625e-05
4.76837158203125e-05
2.384185791015625e-05
1.1920928955078125e-05
5.9604644775390625e-06
2.9802322387695312e-06
1.4901161193847656e-06
7.450580596923828e-07
3.725290298461914e-07
1.862645149230957e-07
9.313225746154785e-08
4.6566128730773926e-08
2.3283064365386963e-08
1.1641532182693481e-08
5.820766091346741e-09
2.9103830456733704e-09
1.4551915228366852e-09
7.275957614183426e-10
3.637978807091713e-10
1.8189894035458565e-10
9.094947017729282e-11
4.547473508864641e-11
2.2737367544323206e-11
1.1368683772161603e-11
5.6843418860808015e-12
2.8421709430404007e-12
1.4210854715202004e-12
7.105427357601002e-13
3.552713678800501e-13
1.7763568394002505e-13
8.881784197001252e-14
4.440892098500626e-14
2.220446049250313e-14
1.1102230246251565e-14
5.551115123125783e-15
2

In [673]:
print(len(selection_prob))

num_list = list(range(1,(num_pop+1)))
selected_pop = np.random.choice(num_list, 2, p=selection_prob, replace=False)



100


ValueError: probabilities contain NaN